In [2]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r ../requirements.txt

import PETs_Experiment

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [4]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 1 # 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   38   Private   89814   HS-grad                9  Married-civ-spouse   

        occupation relationship   race gender  capital-gain  capital-loss  \
0  Farming-fishing      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              50  United-States  <=50K  


In [5]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        ,outlier_columns = {'ignore': ['hours-per-week']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        ,scaling_method  = 'Standard'
        ,scaling_columns = {'focus' : 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Outlierist (IQR): Dropped   224 rows on educational-num. Kept [3.0, 19.0] only.


Preprocessor - Outlierist (IQR): Dropped  1050 rows on fnlwgt         . Kept [-63601.875, 419047.125] only.
Preprocessor - Outlierist (IQR): Dropped  1741 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   207 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3051 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6074 in 36196 rows.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Co

In [6]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.028 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.8822 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30122 rows (same as raw) in 3.2704 sec.
   age  workclass    fnlwgt  education  educational-num  marital-status  \
0   43          4  0.126817          7               12               6   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0          12             4     4       1             0             0   

   hours-per-week  native-country  income  
0              44              39       0  


In [7]:
postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        ,encoder = getattr(preproc ,'encoder' ,None)
                                        ,scaler  = getattr(preproc ,'scaler'  ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
   age         workclass         fnlwgt  education  educational-num  \
0   43  Self-emp-not-inc  191101.890253  Assoc-voc               12   

  marital-status    occupation relationship   race gender  capital-gain  \
0        Widowed  Tech-support    Unmarried  White   Male             0   

   capital-loss  hours-per-week native-country income  
0             0              44     Yugoslavia  <

In [8]:
eval = PETs_Experiment.Evaluator(evaluating_method = 'anonymeter-linkability'
                                 ,data = {'ori'      : splitter.data[1]['train'     ]
                                          ,'syn'     : postproc.data
                                          ,'control' : splitter.data[1]['validation']
                                 }
                                 ,n_neighbors = 10
                                 ,n_jobs      = -1
                                 ,aux_cols = [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
                                             ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
                                             ]
                                 ).eval()

TypeError: PETs_Experiment.Evaluator.Anonymeter.AnonymeterFactory.AnonymeterFactory() got multiple values for keyword argument 'evaluating_method'

In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment


para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 30
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run_single_process()
executor.run()


for k ,v in executor.loader.items():
    print(k)
    print(v.data.head(1))

Executor - Loader: [Adt Income] adult.csv loading time: 10.7713 sec.
Executor - Splitter: Default splitting time: 1.6241 sec.
Preprocessor - Outlierist (IQR): Dropped  1051 rows on fnlwgt         . Kept [-62982.0, 418378.0] only.
Preprocessor - Outlierist (IQR): Dropped  1754 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   218 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3027 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   226 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6073 in 36203 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column race            been labelized

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.0938 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30130 rows (same as raw) in 1.3451 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 19.8852 sec.
Preprocessor - Outlierist (IQR): Dropped  1077 rows on fnlwgt         . Kept [-63094.75, 418955.25] only.
Preprocessor - Outlierist (IQR): Dropped  1738 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   209 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3042 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   243 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6112 in 36151 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 13.2118 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30039 rows (same as raw) in 1.5238 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 14.7719 sec.
Preprocessor - Outlierist (IQR): Dropped  1064 rows on fnlwgt         . Kept [-63231.5, 418172.5] only.
Preprocessor - Outlierist (IQR): Dropped  1708 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   224 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  2988 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   248 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6027 in 36201 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor -

...

Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 15.4647 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30106 rows (same as raw) in 1.4318 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 16.9336 sec.
Preprocessor - Outlierist (IQR): Dropped  1064 rows on fnlwgt         . Kept [-62902.0, 418450.0] only.
Preprocessor - Outlierist (IQR): Dropped  1716 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   213 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  2982 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1245 rows on educational-num. Kept [4.5, 16.5] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6854 in 36161 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor -

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 15.7127 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 29307 rows (same as raw) in 1.4816 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 17.2292 sec.
====== ====== ====== ====== ====== ======
Executor: Total execution time: 528.8497 sec.
====== ====== ====== ====== ====== ======


Loading:   0%|          | 0/1 [00:00<?, ?it/s]




Loading: 100%|██████████| 1/1 [00:12<00:00, 12.74s/it]

In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment

para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        ,'NHANES' : {'filepath'     : '../[sunset]/data/[NHANES] B.csv'
                    ,'header_exist' : False
                    ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 30
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 30
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    # ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    # ,'Preprocessor_scaling'         : True
    # ,'Preprocessor_scaling_method'  : 'Standard'
    # ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
       ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
    #    ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
    #    ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()
    

Loading:   0%|          | 0/2 [00:00<?, ?it/s]




Loading:  50%|█████     | 1/2 [00:15<00:15, 15.39s/it]

Loading: 100%|██████████| 2/2 [01:13<00:00, 40.73s/it]

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0219 sec.Synthesizer (SDV - SingleTable): Metafile loading time: 0.0313 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.

Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 2.3922 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 99 rows (same as raw) in 0.1695 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 13.9484 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN spent 26.5661 sec.


Synthesizer (SDV - SingleTable): Sampling CoupulaGAN # 99 rows (same as raw) in 0.2521 sec.
Executor - Synthesizer: CoupulaGAN synthesizing time: 38.2046 sec.


Loading: 100%|██████████| 2/2 [05:23<00:00, 161.59s/it]


====== ====== ====== ====== ====== ======
Executor: Total execution time: 323.8127 sec.
====== ====== ====== ====== ====== ======
